In [2]:
import pandas as pd
import numpy as np
import torch
from psutil import boot_time

In [5]:
path = 'ml-latest-small/'
ratings = pd.read_csv(path + 'ratings.csv')
movies = pd.read_csv(path + 'movies.csv')
links = pd.read_csv(path + 'links.csv')
tags = pd.read_csv(path + 'tags.csv')

In [7]:
for df in [ratings]:
    print(df.head())
    print("==================================")

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [8]:
for df in [movies]:
    print(df.head())
    print("==================================")

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [9]:
for df in [links]:
    print(df.head())
    print("==================================")

   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0


In [11]:
for df in [tags]:
    print(df.head())
    print("==================================")

   userId  movieId              tag   timestamp
0       2    60756            funny  1445714994
1       2    60756  Highly quotable  1445714996
2       2    60756     will ferrell  1445714992
3       2    89774     Boxing story  1445715207
4       2    89774              MMA  1445715200


In [12]:
print(ratings.shape)

(100836, 4)


In [13]:
print(ratings)

        userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
2            1        6     4.0   964982224
3            1       47     5.0   964983815
4            1       50     5.0   964982931
...        ...      ...     ...         ...
100831     610   166534     4.0  1493848402
100832     610   168248     5.0  1493850091
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352
100835     610   170875     3.0  1493846415

[100836 rows x 4 columns]


In [22]:
# check for gpu availability
print(torch.cuda.is_available())

True


In [23]:
# name of the device to use instead of 'cpu'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
d = np.array([1, 0, 0])
print(d)
print(~d)
print(1 - d)

[1 0 0]
[-2 -1 -1]
[0 1 1]


In [4]:
d = np.array([1, 2 ,3])
print(np.expand_dims(d, axis=0).shape)
print(np.expand_dims(d, axis=1).shape)

(1, 3)
(3, 1)


In [5]:
# Normalize the column of the user-movie matrix (per user)
def z_score_column_normalization(column):

    non_null_ratings = column[~np.isnan(column)]

    if len(non_null_ratings) > 1:
        mean = np.mean(non_null_ratings)
        std = np.std(non_null_ratings)
        if std == 0:
            std = 1

    else:
        mean = np.mean(non_null_ratings) if len(non_null_ratings) > 0 else 0
        std = 1

    normalized_column = (column - mean) / std

    return normalized_column, mean, std

def normalize_and_fill_user_movie_matrix(masked_matrix):
    # apply - axis=0 means that we will normalize each column.
    # normalized_matrix = pd.DataFrame(masked_matrix).apply(z_score_column_normalization, axis=0)

    # fill the missing values with 0.
    df_masked_matrix = pd.DataFrame(masked_matrix)
    normalized_matrix = pd.DataFrame(index=df_masked_matrix.index, columns=df_masked_matrix.columns)
    user_mean = []
    user_std = []

    for col in df_masked_matrix.columns:
        normalized_column, mean, std = z_score_column_normalization(df_masked_matrix[col])
        normalized_matrix[col] = normalized_column
        user_mean.append(mean)
        user_std.append(std)

    normalized_matrix_filled = normalized_matrix.fillna(0)

    return normalized_matrix_filled, user_mean, user_std


In [9]:
# test the above function
user_movie_matrix = pd.DataFrame([[1, 2, 3], 
                                  [4, 5, np.nan],
                                  [7, np.nan, 9],
                                  [np.nan, 10, 11]])

print(user_movie_matrix)

normalized_matrix, user_mean, user_std = normalize_and_fill_user_movie_matrix(user_movie_matrix)

print(normalized_matrix)
print(user_mean)
print(user_std)

     0     1     2
0  1.0   2.0   3.0
1  4.0   5.0   NaN
2  7.0   NaN   9.0
3  NaN  10.0  11.0
          0         1         2
0 -1.224745 -1.111168 -1.372813
1  0.000000 -0.202031  0.000000
2  1.224745  0.000000  0.392232
3  0.000000  1.313198  0.980581
[4.0, 5.666666666666667, 7.666666666666667]
[2.449489742783178, 3.2998316455372216, 3.39934634239519]


In [10]:
# denormalize the prediction matrix
def denormalize_ratings(prediction_matrix, user_means, user_stds):
    means_expanded = np.expand_dims(user_means, axis=0)
    stds_expanded = np.expand_dims(user_stds, axis=0)

    denormalized_matrix = (prediction_matrix * stds_expanded) + means_expanded

    return denormalized_matrix


In [11]:
# test the above function

denorm_ratings = denormalize_ratings(normalized_matrix, user_mean, user_std)

print(denorm_ratings)

     0          1          2
0  1.0   2.000000   3.000000
1  4.0   5.000000   7.666667
2  7.0   5.666667   9.000000
3  4.0  10.000000  11.000000


In [1]:
print(chr(sum(range(ord(min(str(not())))))))

ඞ


In [15]:
d = np.array([0, 1, 2, 3, np.nan])

print(~np.isnan(d))

# separate the non-null values
non_null_ratings = d[~np.isnan(d)]
print(non_null_ratings)

# get only the non-zero values

non_zero_ratings = non_null_ratings[non_null_ratings != 0]
print(non_zero_ratings)


[ True  True  True  True False]
[0. 1. 2. 3.]
[1. 2. 3.]


In [32]:
bitmask = np.array([[0, 1, 1, 0],
                    [1, 0, 0, 1],
                    [0, 1, np.nan, 1]])

print(bitmask)

positions_of_ones = np.where((bitmask != 0) & (~np.isnan(bitmask)))
print(bitmask[positions_of_ones])

[[ 0.  1.  1.  0.]
 [ 1.  0.  0.  1.]
 [ 0.  1. nan  1.]]
[1. 1. 1. 1. 1. 1.]


In [36]:
n2d = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])

res = []

for i in range(n2d.shape[0]):
    res.extend(n2d[i])
    
print(res)
print(type(res[0]))

[1, 2, 3, 4, 5, 6, 7, 8, 9]
<class 'numpy.int64'>
